In [44]:
import cv2
import os
import tensorflow as tf
import numpy as np
import imageio
import gdown
import dlib
from typing import List
from matplotlib import pyplot as plt

In [45]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass

Using some of the same ideas as presented in the paper, we process a video with the dlib face detection and tracking.
With the mouth-centered image landmark, 

In [46]:
#Preprocessing: All videos are 3 seconds long with a frame rate of 25fps. 
#The videos were processed with the DLib face detector, and the iBug face landmark predictor (Sagonas et al., 2013) with 68 landmarks coupled with an online Kalman Filter. 
#Using these landmarks, we apply an affine transformation to extract a mouth-centred crop of size 100 × 50 pixels per frame. 
#We standardise the RGB channels over the whole training set to have zero mean and unit variance.
def preprocess_video(path: str) -> List[float]:
    cap = cv2.VideoCapture(path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Convert the frame to grayscale 
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Detect faces in the frame
        faces = hog_face_detector(frame)
        # If no face is detected, move to the next frame
        if len(faces) == 0:
            continue
        # Get the first face
        face = faces[0]
        # Get the landmarks
        landmarks = dlib_facelandmark(frame, face)
        mouth = landmarks.parts()[48:68]
        mouth = np.array([[point.x, point.y] for point in mouth])
        mouth = mouth.astype(np.float32)
        mouth = mouth.reshape(-1)
        # crop the mouth from the frame
        frame = frame[mouth[1]:mouth[7], mouth[0]:mouth[6]]
        # Resize the frame
        frame = cv2.resize(frame, (100, 50))

        frames.append(mouth)
    cap.release()

    # Standardize
    mean = frames - frames.mean()
    std = frames.std()
    frames = (frames - mean) / std
    return frames

In [47]:
vocab = [l for l in "abcdefghijklmnopqrstuvwxyz'!?1234567890 "] # 39 characters

In [48]:
preprocess_video('data/s1/bbaf2n.mpg')

error: OpenCV(4.9.0) /Users/xperience/GHA-OpenCV-Python2/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/shapedescr.cpp:874: error: (-215:Assertion failed) npoints >= 0 && (depth == CV_32F || depth == CV_32S) in function 'pointSetBoundingRect'
